In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os


In [ ]:
stations_txt_path = "/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Data/GIS/stations_statics.txt"
shapefile_path = "/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Data/GIS/polygon_PYRENEES.shp"
output_csv_path = "/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Data/GIS/stations_statics_pyrenees.csv"
output_map_path = "/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Output/stations_pyrenees_statics.png"

# Créer dossier de sortie si nécessaire
output_dir = os.path.dirname(output_map_path)
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Lecture du fichier .txt et parsing des stations
stations = []
current_station = {}
state = "WAITING_FOR_NAME"

regex_name = re.compile(r"^\s*([A-Za-zÀ-ÖØ-öø-ÿ\s\-'’]+?)\s*$")
regex_altitude_value = re.compile(r"^\s*(\d+)\s*mètres\s*$")
regex_coords_value = re.compile(r"^\s*([\d,\.]+°[NS])\s*\|\s*([\d,\.]+°[OE])\s*$")

with open(stations_txt_path, "r", encoding="ISO-8859-1") as f:
    for line in f:
        line = line.strip()
        if not line:
            if current_station and "Nom_Station" in current_station and "Latitude" in current_station and "Longitude" in current_station and "Altitude_m" in current_station:
                stations.append(current_station)
            current_station = {}
            state = "WAITING_FOR_NAME"
            continue

        if state == "WAITING_FOR_NAME":
            match = regex_name.match(line)
            if match:
                current_station = {"Nom_Station": match.group(1).strip(), "Code_ID": ""}
                state = "WAITING_FOR_ALTITUDE_LABEL"

        elif state == "WAITING_FOR_ALTITUDE_LABEL":
            if line == "Altitude":
                state = "WAITING_FOR_ALTITUDE_VALUE"

        elif state == "WAITING_FOR_ALTITUDE_VALUE":
            match = regex_altitude_value.match(line)
            if match:
                current_station["Altitude_m"] = int(match.group(1))
                state = "WAITING_FOR_COORD_LABEL"

        elif state == "WAITING_FOR_COORD_LABEL":
            if line == "Coordonnées":
                state = "WAITING_FOR_COORD_VALUE"

        elif state == "WAITING_FOR_COORD_VALUE":
            match = regex_coords_value.match(line)
            if match:
                lat_str = match.group(1).replace(",", ".")
                lon_str = match.group(2).replace(",", ".")
                lat = float(lat_str[:-2]) * (-1 if 'S' in lat_str else 1)
                lon = float(lon_str[:-2]) * (-1 if 'O' in lon_str else 1)
                current_station["Latitude"] = lat
                current_station["Longitude"] = lon
                stations.append(current_station)
                current_station = {}
                state = "WAITING_FOR_NAME"

df_stations = pd.DataFrame(stations)
print(df_stations.head())


In [ ]:
# Conversion en GeoDataFrame
gdf_stations = gpd.GeoDataFrame(
    df_stations,
    geometry=gpd.points_from_xy(df_stations["Longitude"], df_stations["Latitude"]),
    crs="EPSG:4326"
)


In [ ]:
# Chargement du shapefile des Pyrénées
gdf_pyrenees = gpd.read_file(shapefile_path)
if gdf_pyrenees.crs != gdf_stations.crs:
    gdf_pyrenees = gdf_pyrenees.to_crs(gdf_stations.crs)


In [ ]:
# Visualisation de toutes les stations
fig, ax = plt.subplots(figsize=(12, 10))
gdf_pyrenees.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.7, label='Pyrénées')
gdf_stations.plot(ax=ax, marker='o', color='blue', markersize=10, label='Toutes les stations')
ax.set_title("Toutes les stations et le polygone des Pyrénées")
ax.legend()
plt.show()


In [ ]:
# Filtrage des stations à l’intérieur du polygone
gdf_filtered = gpd.sjoin(gdf_stations, gdf_pyrenees, how="inner", predicate="within")
gdf_filtered = gdf_filtered.drop(columns=["index_right"], errors="ignore")


In [ ]:
# Sauvegarde du CSV filtré
gdf_filtered.drop(columns="geometry").to_csv(output_csv_path, index=False)
print(f" Fichier CSV sauvegardé : {output_csv_path}")


In [ ]:
#Carte finale des stations filtrées

fig, ax = plt.subplots(figsize=(12, 10))
gdf_pyrenees.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.7)
gdf_filtered.plot(ax=ax, marker='o', color='red', markersize=50, label='Stations filtrées')
ax.set_title("Stations météo dans les Pyrénées")
plt.legend()
plt.savefig(output_map_path, dpi=300, bbox_inches='tight')
plt.show()
print(f" Carte PNG sauvegardée : {output_map_path}")
